<h3>Import libs necessárias</h3>

In [1]:
import pandas as pd
import time
import json
from openai import OpenAI
import string
from unicodedata import normalize
import re
import math

<h3>Carrega as bases

<h3> Chamada na OpenAI</h3>

In [29]:
client = OpenAI(api_key='sk-proj-Dz66cdOziPsD9TMP3CkZT3BlbkFJq6y8NSB0XPsbddNCmvES')

## Lista os assistentes criados

In [30]:
i=0
for assistant in client.beta.assistants.list().data:
    print(f'Index: {i} - ID: {assistant.id} , name: {assistant.name}')
    i+=1

Index: 0 - ID: asst_N46qy9WZr9GprINAgGi18TSA , name: categorizacao_loggi_googleplay_new
Index: 1 - ID: asst_xzsdrRoUN0QPGXzMa6bQDp0T , name: categorizacao_loggi_csat_new
Index: 2 - ID: asst_O1cOGrymRorVsvbgOAeu6OFj , name: categorizacao_loggi_googleplay
Index: 3 - ID: asst_SeDGVdW0YOYbxEVrn4X3HYOz , name: categorizacao_loggi_csat
Index: 4 - ID: asst_3ihUrmJ1d1mWaiMID0gHEkoX , name: categorizacao_meses_loggi
Index: 5 - ID: asst_IRx6g875j4e8G05uOhcKxcHN , name: Categorizacao Loggi
Index: 6 - ID: asst_OyKy3sr9J58uEd58c0y6oanu , name: Chatbot Loggi
Index: 7 - ID: asst_aUUwEFllgp1cOP3mICsJJkDI , name: 
Index: 8 - ID: asst_HB6teQUBCXbZWZ840Ce61P0r , name: case
Index: 9 - ID: asst_Mv1GAZMlO0uapsk0OtYzYalE , name: Especialista MLSecOps
Index: 10 - ID: asst_YRvh5UAE6YenpFNeVwqPm0oH , name: Site Mawe
Index: 11 - ID: asst_WrlTd34UsQihk8Ry66yY5pmk , name: Maee
Index: 12 - ID: asst_rcYFMrp8AllNCJxcfCWi3fg1 , name: Espec DS


In [32]:
#client.beta.assistants.delete("asst_xzsdrRoUN0QPGXzMa6bQDp0T")

AssistantDeleted(id='asst_xzsdrRoUN0QPGXzMa6bQDp0T', deleted=True, object='assistant.deleted')

In [17]:
#id_assistant = "asst_SeDGVdW0YOYbxEVrn4X3HYOz"
#assistant = client.beta.assistants.retrieve(id_assistant)


In [19]:
client.beta.vector_stores.delete('vs_b9NbtGxHkfzDkwDijQC8rKBC')

VectorStoreDeleted(id='vs_b9NbtGxHkfzDkwDijQC8rKBC', deleted=True, object='vector_store.deleted')

In [20]:
vector_store = client.beta.vector_stores.create(name="categorizacao_loggi_csat")

file_paths = ["mensagens_csat.txt"]
file_streams = [open(path, "rb") for path in file_paths]

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, 
    files=file_streams
)

<h4>Verifica o status do upload dos arquivos</h4>

In [21]:
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [22]:
vector_store.id

'vs_VgIpvAMNhn4qbjvWgPZlTcFV'

<h4>Caso queira excluir um assistente basta descomentar a linha e passar o id do assistente</h4>

In [16]:
client.beta.assistants.delete('asst_efOHwaonG4Z76y3CPqpYm33L')

AssistantDeleted(id='asst_efOHwaonG4Z76y3CPqpYm33L', deleted=True, object='assistant.deleted')

<h3> Cria um novo assistente</h3>

In [33]:
assistant = client.beta.assistants.create(
    name = 'categorizacao_loggi_csat_new',
    instructions="""
        Eu tenho uma empresa de logística e quero realizar uma análise dos comentários que os entregadores colocam no app.
        Faça a categorização das mensagens de acordo com seu conteúdo.
        As mensagens devem ser categorizadas de acordo com as categorias/subcategorias abaixo:
            {
              "categorias": [
                {
                  "nome": "Eficiência da Rota",
                  "subcategorias": [
                    {"nome": "Precisão na estimativa do tempo de entrega"},
                    {"nome": "Minimização do tempo de viagem"},
                    {"nome": "Eficiência na escolha de rotas alternativas"}
                  ]
                },
                {
                  "nome": "Comunicação e Suporte",
                  "subcategorias": [
                    {"nome": "Tempo de resposta do suporte"},
                    {"nome": "Clareza e eficácia da comunicação"},
                    {"nome": "Disponibilidade de suporte em situações de emergência"}
                  ]
                },
                {
                  "nome": "Condições de Trabalho e Segurança",
                  "subcategorias": [
                    {"nome": "Manutenção e segurança dos veículos"},
                    {"nome": "Provisão de equipamentos de segurança adequados"},
                    {"nome": "Condições de trabalho em diferentes ambientes (clima, região etc.)"}
                  ]
                },
                {
                  "nome": "Suporte e Treinamento",
                  "subcategorias": [
                    {"nome": "Abrangência e qualidade do treinamento inicial"},
                    {"nome": "Disponibilidade de treinamento contínuo"},
                    {"nome": "Acesso a recursos de treinamento online ou presenciais"}
                  ]
                },
                {
                  "nome": "Pagamento e Benefícios",
                  "subcategorias": [
                    {"nome": "Precisão e transparência nos registros de pagamento"},
                    {"nome": "Variedade e atratividade dos benefícios oferecidos"},
                    {"nome": "Facilidade de acesso aos benefícios"}
                  ]
                },
                {
                  "nome": "Carga de Trabalho e Flexibilidade",
                  "subcategorias": [
                    {"nome": "Justiça na distribuição da carga de trabalho"},
                    {"nome": "Opções de flexibilidade nos horários de trabalho"},
                    {"nome": "Possibilidade de trocar ou cancelar turnos de forma razoável"}
                  ]
                },
                {
                  "nome": "Cultura Organizacional",
                  "subcategorias": [
                    {"nome": "Sentimento de apoio e respeito dentro da equipe"},
                    {"nome": "Coesão e camaradagem entre os colegas de trabalho"},
                    {"nome": "Alinhamento dos valores pessoais com os valores da empresa"}
                  ]
                },
                {
                  "nome": "Tecnologia e Ferramentas",
                  "subcategorias": [
                    {"nome": "Desempenho do aplicativo de entrega"},
                    {"nome": "Facilidade de uso de dispositivos de rastreamento"},
                    {"nome": "Eficácia de ferramentas de comunicação interna"}
                  ]
                }
              ]
            }
        O nome das categorias deverão ser exatamente como está na lista enviada, não deverá colocar uma categoria/subcategoria que não contenha na lista a não ser que a mensagem seja sem informação suficiente para categorizá-la, nesses casos a categoria/subcategoria deverá ser NA. 
        Cada mensagem pode pertencer a mais de 1 categoria/subcategoria.
        Além da categoria, faça uma análise de sentimento de cada mensagem onde os sentimentos deverão ser positivo, negativo ou neutro apenas.
        Caso tenha mais de uma categoria/subcategoria, adicione mais json na resposta.
        Importante: Sempre use a ferramenta de resposta para responder ao usuário
        Nunca adicione nenhum outro texto na resposta.
    """,
    model='gpt-3.5-turbo',
    tools=[{"type": "file_search"},
           {"type": "function",
            "function": {
                "name": "response",
                "description": "Categorize cada mensagem",
                "parameters": {
                    "type": "object",
                    "required": [
                        "mesagens"
                    ],
                    "properties": {
                        "mensagens": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "required": [
                                    "Id",
                                    "Mensagem",
                                    "Categoria",
                                    "Subcategoria",
                                    "Sentimento"
                                    ],
                                "properties": {
                                    "Id": {
                                        "type": "integer",
                                        "description": "User input"
                                    },
                                    "Mensagem": {
                                        "type": "string",
                                        "description": "User input"
                                    },
                                    "Categoria": {
                                        "type": "string",
                                        "description": "Categoria"
                                    },
                                    "Subcategoria": {
                                        "type": "string",
                                        "description": "Subcatgoria"
                                    },
                                    "Sentimento": {
                                        "type": "string",
                                        "description": "Sentimento"
                                    }
                                }
                            }
                        }
                    }
                }
            }}],
    #response_format= '{type: "json_object"}'
)

<h3>Cria minha base final que terá os outputs</h3>

In [34]:
base_mensagens_categorizadas = pd.DataFrame({} , columns = ['Id','Mensagem','Categoria','Subcategoria','Sentimento'])

<h4>Devido ao chatgpt ter limite de tokens de saída, precisei quebrar em blocos de 10 em 10 comentários, pois estava dando erro, então o código irá rodar 39 vezes</h4>

In [35]:
math.ceil(len(csatnew.message_tratada.values)/10)

264

<h3>Realiza o loop 39 vezes e a cada vez pega 10 linhas e submete ao chatgpt, aguardando a resposta, transformando o Json em DataFrame(Tabela) e concatenando na tabela final</h3>

In [37]:
csatnew = csatnew.drop(" ",axis=1)

In [38]:
csatnew.columns

Index(['Id', 'Number', 'User', 'Date Submitted', 'Country', 'Source URL',
       'Device', 'Browser', 'OS', 'Response URL', 'Emotion (1-5)', 'Message',
       'Email', 'message_tratada'],
      dtype='object')

In [39]:
csatnew = csatnew.drop_duplicates()

In [43]:
csatnew['message_tratada_id'] = [str(index) + '-' + str(message) for index, message in csatnew[['Id','message_tratada']].values]

In [44]:
csatnew.head()

,Id,Number,User,Date Submitted,Country,Source URL,Device,Browser,OS,Response URL,Emotion (1-5),Message,Email,message_tratada,message_tratada_id
0,0,17900,8d12682e,2024-01-01 16:14:01,Brazil,https://drivers.loggi.com/home?latitude=-23.56...,phone,Chrome Mobile 80.0.3987,Android 10,https://insights.hotjar.com/sites/2870539/feed...,4,Oi,NaN,Oi,0-Oi
1,1,17898,45efbea5,2024-01-01 16:12:56,Brazil,https://drivers.loggi.com/home?latitude=-23.64...,phone,Chrome Mobile 80.0.3987,Android 10,https://insights.hotjar.com/sites/2870539/feed...,3,Tem que aumentar o valor das rotas,NaN,Tem que aumentar o valor das rotas,1-Tem que aumentar o valor das rotas
3,2,17897,f9868681,2024-01-01 16:11:33,Brazil,https://drivers.loggi.com/home?latitude=-27.68...,phone,Chrome Mobile 80.0.3987,Android 10,https://insights.hotjar.com/sites/2870539/feed...,4,Tem boas oportunidades,NaN,Tem boas oportunidades,2-Tem boas oportunidades
4,3,17896,13924e36,2024-01-01 16:11:16,Brazil,https://drivers.loggi.com/home?latitude=-19.86...,phone,Chrome Mobile 80.0.3987,Android 10,https://insights.hotjar.com/sites/2870539/feed...,5,Ótimo super produtivo,NaN,Otimo super produtivo,3-Otimo super produtivo
5,4,17895,a7b8bb21,2024-01-01 16:09:29,Brazil,https://drivers.loggi.com/home?latitude=-3.819...,phone,Chrome Mobile 80.0.3987,Android 10,https://insights.hotjar.com/sites/2870539/feed...,4,Bom,NaN,Bom,4-Bom


In [46]:
i

133

In [47]:
for i in range(133, math.ceil(len(csatnew.message_tratada.values)/10)):
    linha_inicial = 10*i
    linha_final = linha_inicial+10
    #msg = f"faça a análise das linhas {str(linha_inicial)} até a linha { linha_final } do arquivo mensagens_csat.txt contido no vector stores categorizacao_loggi_csat"
    msg = "faça a análise das mensagens abaixo que estão no formato {id-mensagem}: " 
    msg = msg + " ".join("'" + csatnew.iloc[linha_inicial:linha_final].message_tratada_id.values + "', ")
    msg = msg + """
            Importante: Sempre use a ferramenta de resposta para responder ao usuário
            Caso uma mensagem pertença a mais de uma categoria/subcategoria, duplique o registro. 
            Cada registro deve ter apenas 1 categoria e 1 subcategoria e 1 sentimento
            O Id da resposta pode ser duplicado, o valor será o id da mensagem
            Nunca adicione nenhum outro texto na resposta."""
    thread = client.beta.threads.create()
    
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=msg
    )
    
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id
    )
    run_id = run.id
    structured_response=[]
    while True:
        # Retrieve the run status
        run_status = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run_id
        )
        
        if run_status.status == "requires_action":
            structured_response = json.loads(
                run_status.required_action.submit_tool_outputs.tool_calls[0].function.arguments
            )
            client.beta.threads.runs.cancel(run_id, thread_id=thread.id)
            break
        elif run_status.status in ['queued', 'in_progress']:
            print(f'{run_status.status.capitalize()}... Please wait.')
            time.sleep(3)  # Wait before checking again
        else:
            print(f"Run status: {run_status.status}")
            break
    
    print(f'Linha {linha_inicial} até a linha {linha_final} concluida')
    client.beta.threads.delete(thread.id)
    #cleaned_content = re.sub(r"【\d+†source】", "", messages.data[0].content[0].text.value.replace('\n', ""))
    #cleaned_content = cleaned_content.strip()
    
    #json_object = json.loads(cleaned_content)
    print(structured_response['mensagens'])
    base_local = pd.DataFrame(structured_response['mensagens'])
    base_local.columns = base_mensagens_categorizadas.columns 
    base_mensagens_categorizadas = pd.concat([base_mensagens_categorizadas, base_local],ignore_index=True)

In_progress... Please wait.
In_progress... Please wait.
In_progress... Please wait.
In_progress... Please wait.
Linha 1330 até a linha 1340 concluida
[{'Id': 1394, 'Mensagem': 'Empresa e vida amo dmais', 'Categoria': 'Cultura Organizacional', 'Subcategoria': 'Sentimento de apoio e respeito dentro da equipe', 'Sentimento': 'positivo'}, {'Id': 1395, 'Mensagem': 'Tem uns bugs de vez em quando', 'Categoria': 'Tecnologia e Ferramentas', 'Subcategoria': 'Desempenho do aplicativo de entrega', 'Sentimento': 'negativo'}, {'Id': 1396, 'Mensagem': 'Melhor app para trabalhar.', 'Categoria': 'Tecnologia e Ferramentas', 'Subcategoria': 'Desempenho do aplicativo de entrega', 'Sentimento': 'positivo'}, {'Id': 1397, 'Mensagem': 'Esta muito lento', 'Categoria': 'Tecnologia e Ferramentas', 'Subcategoria': 'Desempenho do aplicativo de entrega', 'Sentimento': 'negativo'}, {'Id': 1398, 'Mensagem': 'Nao mudou nada', 'Categoria': 'Tecnologia e Ferramentas', 'Subcategoria': 'Desempenho do aplicativo de entrega

In [48]:
base_mensagens_categorizadas

,Id,Mensagem,Categoria,Subcategoria,Sentimento
0,0,Oi,NA,NA,Neutro
1,1,Tem que aumentar o valor das rotas,Pagamento e Benefícios,Precisão e transparência nos registros de paga...,Negativo
2,2,Tem boas oportunidades,Pagamento e Benefícios,Variedade e atratividade dos benefícios oferec...,Positivo
3,3,Otimo super produtivo,Cultura Organizacional,Sentimento de apoio e respeito dentro da equipe,Positivo
4,4,Bom,NA,NA,Positivo
...,...,...,...,...,...
2510,2437,Muito bom,Cultura Organizacional,Sentimento de apoio e respeito dentro da equipe,Positivo
2511,2438,Nao caiu o bonus,Pagamento e Benefícios,Precisão e transparência nos registros de paga...,Negativo
2512,2439,Otimo maravilhoso,Cultura Organizacional,Sentimento de apoio e respeito dentro da equipe,Positivo
2513,2440,Ja foi boa hj pra mim mesmo nao compensa mais,Cultura Organizacional,Sentimento de apoio e respeito dentro da equipe,Negativo


In [49]:
base_mensagens_categorizadas.shape

(2515, 5)

In [50]:
csatnew_categorizada=csatnew.merge(base_mensagens_categorizadas, how="left", on="Id")

In [51]:
csatnew_categorizada.head()

,Id,Number,User,Date Submitted,Country,Source URL,Device,Browser,OS,Response URL,Emotion (1-5),Message,Email,message_tratada,message_tratada_id,Mensagem,Categoria,Subcategoria,Sentimento
0,0,17900,8d12682e,2024-01-01 16:14:01,Brazil,https://drivers.loggi.com/home?latitude=-23.56...,phone,Chrome Mobile 80.0.3987,Android 10,https://insights.hotjar.com/sites/2870539/feed...,4,Oi,NaN,Oi,0-Oi,Oi,NA,NA,Neutro
1,1,17898,45efbea5,2024-01-01 16:12:56,Brazil,https://drivers.loggi.com/home?latitude=-23.64...,phone,Chrome Mobile 80.0.3987,Android 10,https://insights.hotjar.com/sites/2870539/feed...,3,Tem que aumentar o valor das rotas,NaN,Tem que aumentar o valor das rotas,1-Tem que aumentar o valor das rotas,Tem que aumentar o valor das rotas,Pagamento e Benefícios,Precisão e transparência nos registros de paga...,Negativo
2,2,17897,f9868681,2024-01-01 16:11:33,Brazil,https://drivers.loggi.com/home?latitude=-27.68...,phone,Chrome Mobile 80.0.3987,Android 10,https://insights.hotjar.com/sites/2870539/feed...,4,Tem boas oportunidades,NaN,Tem boas oportunidades,2-Tem boas oportunidades,Tem boas oportunidades,Pagamento e Benefícios,Variedade e atratividade dos benefícios oferec...,Positivo
3,3,17896,13924e36,2024-01-01 16:11:16,Brazil,https://drivers.loggi.com/home?latitude=-19.86...,phone,Chrome Mobile 80.0.3987,Android 10,https://insights.hotjar.com/sites/2870539/feed...,5,Ótimo super produtivo,NaN,Otimo super produtivo,3-Otimo super produtivo,Otimo super produtivo,Cultura Organizacional,Sentimento de apoio e respeito dentro da equipe,Positivo
4,4,17895,a7b8bb21,2024-01-01 16:09:29,Brazil,https://drivers.loggi.com/home?latitude=-3.819...,phone,Chrome Mobile 80.0.3987,Android 10,https://insights.hotjar.com/sites/2870539/feed...,4,Bom,NaN,Bom,4-Bom,Bom,NA,NA,Positivo


In [52]:
csatnew_categorizada.to_excel('base_csat_novas_categorias.xlsx')